<a href="https://colab.research.google.com/github/ikc-arya/AIR_SS25/blob/teo/bm25_best_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)



- **https://www.nltk.org/**
- **https://spacy.io/docs**

In [1]:
!pip install nltk spacy pandarallel ipywidgets rank_bm25

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=b003bc1bf2e0b6890d80665764c7f6e7af57ac6ec922d1843478498d9c0ef367
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel


In [2]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 57.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [56]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [57]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask_4b/subtask4b_collection_data.pkl'

In [58]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

### Dataframe Information (`df_collection.info()`):

The dataframe `df_collection` contains **7,718 entries** (rows) and **17 columns**. This is the metadata for **7,718 papers** in the CORD-19 dataset.


In [130]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   cord_uid             7718 non-null   object        
 1   source_x             7718 non-null   object        
 2   title                7718 non-null   object        
 3   doi                  7677 non-null   object        
 4   pmcid                4959 non-null   object        
 5   pubmed_id            6233 non-null   object        
 6   license              7718 non-null   object        
 7   abstract             7718 non-null   object        
 8   publish_time         7715 non-null   object        
 9   authors              7674 non-null   object        
 10  journal              6668 non-null   object        
 11  mag_id               0 non-null      float64       
 12  who_covidence_id     528 non-null    object        
 13  arxiv_id             20 non-null 

In [131]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,...,time,timet,normalized_title,normalized_abstract,tokenized_title,tokenized_abstract,cleaned_title,cleaned_abstract,cleaned_authors,authors_last_names
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",...,2008-07-09,1215561600,professional and home-made face masks reduce e...,background: governments are preparing for a po...,"[professional, and, home-made, face, masks, re...","[background:, governments, are, preparing, for...",profession homemad face mask reduc exposur res...,background govern prepar potenti influenza pan...,van der sand teuni sabel,van der Sande Teunis Sabel
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",...,2011-08-16,1313452800,the failure of r (0),"the basic reproductive ratio, r (0), is one of...","[the, failure, of, r, (0)]","[the, basic, reproductive, ratio,, r, (0),, is...",failur r 0,basic reproduct ratio r 0 one fundament concep...,li blakeley smith,Li Blakeley Smith?
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",...,2012-01-01,1325376000,pulmonary sequelae in a patient recovered from...,the pandemic of swine flu (h1n1) influenza spr...,"[pulmonary, sequelae, in, a, patient, recovere...","[the, pandemic, of, swine, flu, (h1n1), influe...",pulmonari sequela patient recov swine flu,pandemic swine flu h1n1 influenza spread invol...,singh sharma patel,Singh Sharma Patel
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",...,2012-11-29,1354147200,what was the primary mode of smallpox transmis...,the mode of infection transmission has profoun...,"[what, was, the, primary, mode, of, smallpox, ...","[the, mode, of, infection, transmission, has, ...",primari mode smallpox transmiss implic biodefens,mode infect transmiss profound implic effect c...,milton,Milton
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",...,2013-02-03,1359849600,"lessons from the history of quarantine, from p...","in the new millennium, the centuries-old strat...","[lessons, from, the, history, of, quarantine,,...","[in, the, new, millennium,, the, centuries-old...",lesson histori quarantin plagu influenza,new millennium centuriesold strategi quarantin...,tognotti,Tognotti


#**Replacing synonyms**

 Using a thesaurus to look up synonyms for words in both the tweets and the scientific articles, and set all these to the same word. Replacing all synonyms with one common word means that there is more tollerance towards too tweets not using the exact same wording as the research article.

In [132]:
# Read the thesaurus file
thesaurus_file = pd.read_csv("./subtask_4b/Thesaurus_covid19_formatted_flattend.csv", delimiter=';', encoding='cp1252')

# Build the thesaurus
thesaurus = {
    row['prefLabel_en']: row['altLabel_en'].strip('[]').split(', ')
    for _, row in thesaurus_file.iterrows()
}

To make lookup easier, the dictonary is also reversed, where data previously stored as the value is now the key and the key becomes the value.

In [133]:
# Reverse the thesaurus for fast lookups
reverse_thesaurus = {}
for main_term, synonyms in thesaurus.items():
    for synonym in synonyms:
        reverse_thesaurus[synonym.lower()] = main_term.lower()
    reverse_thesaurus[main_term.lower()] = main_term.lower()

A function is defined for converting the words in a text to the word that has been set as the value in the dictionary, if the word is in the key. In other words converting all synonyms of the same word to one specific word.

In [134]:
# Normalize the text using the reverse thesaurus
def normalize_text(text, reverse_thesaurus):
    words = text.lower().split()
    normalized_words = [reverse_thesaurus.get(word, word) for word in words]
    return ' '.join(normalized_words)

All documents are passed through the normalize_text function so that synonyms of a word gets replaced with the same one.

In [135]:
# Apply normalization directly to the title and abstract columns
df_collection['normalized_title'] = df_collection['title'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)

df_collection['normalized_abstract'] = df_collection['abstract'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)

df_collection[['title', 'abstract', 'normalized_title', 'normalized_abstract', 'authors']].head()

,title,abstract,normalized_title,normalized_abstract,authors
162,Professional and Home-Made Face Masks Reduce E...,BACKGROUND: Governments are preparing for a po...,professional and home-made face masks reduce e...,background: governments are preparing for a po...,"van der Sande, Marianne; Teunis, Peter; Sabel,..."
611,The Failure of R (0),"The basic reproductive ratio, R (0), is one of...",the failure of r (0),"the basic reproductive ratio, r (0), is one of...","Li, Jing; Blakeley, Daniel; Smith?, Robert J."
918,Pulmonary sequelae in a patient recovered from...,The pandemic of swine flu (H1N1) influenza spr...,pulmonary sequelae in a patient recovered from...,the pandemic of swine flu (h1n1) influenza spr...,"Singh, Virendra; Sharma, Bharat Bhushan; Patel..."
993,What was the primary mode of smallpox transmis...,The mode of infection transmission has profoun...,what was the primary mode of smallpox transmis...,the mode of infection transmission has profoun...,"Milton, Donald K."
1053,"Lessons from the History of Quarantine, from P...","In the new millennium, the centuries-old strat...","lessons from the history of quarantine, from p...","in the new millennium, the centuries-old strat...","Tognotti, Eugenia"


Too include author names in the search for relevant documents, the lastname is extracted from the authors column. The first names are excluded to prevent accidental matches, where a authors could share a first name with authors of the actual article that is referenced by the tweet, without having anything to do with the article.

In [137]:
def extract_last_names(authors_string):
    if pd.isna(authors_string):
        return []
    return ' '.join([author.split(',')[0].strip() for author in authors_string.split(';') if ',' in author])

df_collection['authors_last_names'] = df_collection['authors'].apply(extract_last_names)

df_collection[['title', 'abstract', 'normalized_title', 'normalized_abstract', 'authors_last_names']].head()

,title,abstract,normalized_title,normalized_abstract,authors_last_names
162,Professional and Home-Made Face Masks Reduce E...,BACKGROUND: Governments are preparing for a po...,professional and home-made face masks reduce e...,background: governments are preparing for a po...,van der Sande Teunis Sabel
611,The Failure of R (0),"The basic reproductive ratio, R (0), is one of...",the failure of r (0),"the basic reproductive ratio, r (0), is one of...",Li Blakeley Smith?
918,Pulmonary sequelae in a patient recovered from...,The pandemic of swine flu (H1N1) influenza spr...,pulmonary sequelae in a patient recovered from...,the pandemic of swine flu (h1n1) influenza spr...,Singh Sharma Patel
993,What was the primary mode of smallpox transmis...,The mode of infection transmission has profoun...,what was the primary mode of smallpox transmis...,the mode of infection transmission has profoun...,Milton
1053,"Lessons from the History of Quarantine, from P...","In the new millennium, the centuries-old strat...","lessons from the history of quarantine, from p...","in the new millennium, the centuries-old strat...",Tognotti


Some preprocessing is applied to the title and abstract. The same is also done to the authors column, but without stemming to prevent stemming of names.

In [142]:
import re
import string
import html
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
nltk.download('stopwords')



stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
nlp = spacy.load("en_core_web_md")


important_terms = {
    'covid', 'covid-19', 'hiv', 'rna', 'sars', 'r0', 'h1n1', 'who', 'cdc',
    'micro', 'kg', 'm/s', 'flu', 'cancer', 'aids', 'diabetes', 'malaria',
    'tuberculosis', 'hepatitis', 'pneumonia', 'leprosy', 'arthritis', 'asthma',
    'hypertension', 'obesity', 'influenza', 'hiv', 'hepatitis-b', 'coronavirus',
    'outbreak', 'pandemic', 'endemic', 'vaccine', 'antiviral', 'antibiotic',
    'surgical', 'gene', 'genome', 'mutation', 'pathogen'
}

def preprocess_text(text, terms_extractor=nlp):
    text = re.sub(r'http\S+|www\S+', '', text)

    text = html.unescape(text)

    text = text.lower()

    #Keep numbers as the they are
    #text = ' '.join([word if word in important_terms else re.sub(r'\d+', '', word) for word in text.split()])

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.translate(str.maketrans('', '', string.punctuation))

    text = ' '.join([word for word in text.split() if word not in stop_words or word in important_terms])

    text = ' '.join([stemmer.stem(word) if word not in important_terms else word for word in text.split()])

    #Remove spacy entities
    #doc = terms_extractor(text)
    #entities = ' '.join(ent.text for ent in doc.ents)

    return text



def preprocess_author(text):

    text = text.lower()

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.translate(str.maketrans('', '', string.punctuation))

    text = ' '.join([word for word in text.split() if word not in stop_words or word in important_terms])

    text = ' '.join([stemmer.stem(word) if word not in important_terms else word for word in text.split()])

    return text

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
df_collection['cleaned_title'] = df_collection['normalized_title'].fillna('').parallel_apply(preprocess_text)
df_collection['cleaned_abstract'] = df_collection['normalized_abstract'].fillna('').parallel_apply(preprocess_text)
df_collection['cleaned_authors'] = df_collection['authors_last_names'].fillna('').apply(
    lambda x: ' '.join(x) if isinstance(x, list) else x
).parallel_apply(preprocess_author)

In [143]:
filtered_df = df_collection[['title', 'cleaned_title', 'abstract', 'cleaned_abstract', 'cleaned_authors']]
filtered_df.head(10)

,title,cleaned_title,abstract,cleaned_abstract,cleaned_authors
162,Professional and Home-Made Face Masks Reduce E...,profession homemad face mask reduc exposur res...,BACKGROUND: Governments are preparing for a po...,background govern prepar potenti influenza pan...,van der sand teuni sabel
611,The Failure of R (0),failur r 0,"The basic reproductive ratio, R (0), is one of...",basic reproduct ratio r 0 one fundament concep...,li blakeley smith
918,Pulmonary sequelae in a patient recovered from...,pulmonari sequela patient recov swine flu,The pandemic of swine flu (H1N1) influenza spr...,pandemic swine flu h1n1 influenza spread invol...,singh sharma patel
993,What was the primary mode of smallpox transmis...,primari mode smallpox transmiss implic biodefens,The mode of infection transmission has profoun...,mode infect transmiss profound implic effect c...,milton
1053,"Lessons from the History of Quarantine, from P...",lesson histori quarantin plagu influenza,"In the new millennium, the centuries-old strat...",new millennium centuriesold strategi quarantin...,tognotti
1589,Anxiety and Depression: Linkages with Viral Di...,anxieti depress linkag viral diseas,Anxiety and mood disorders are common in the g...,anxieti mood disord common gener popul countri...,coughlin
2069,Effects of Ultraviolet Germicidal Irradiation ...,effect ultraviolet germicid irradi uvgi n95 re...,The ability to disinfect and reuse disposable ...,abil disinfect reus dispos n95 filter facepiec...,lindsley martin thewli sarkisian nwoko mead noti
2843,Secretome of Intestinal Bacilli: A Natural Gua...,secretom intestin bacilli natur guard patholog,Current studies of human gut microbiome usuall...,current studi human gut microbiom usual consid...,ilinskaya ulyanova yarullina gataullin
2952,Long term outcomes in survivors of epidemic In...,long term outcom survivor epidem influenza h7n...,Patients who survive influenza A (H7N9) virus ...,patient world health organ surviv influenza h7...,chen wu hao yang lu chen li
3044,Far-UVC light: A new tool to control the sprea...,faruvc light new tool control spread airbornem...,Airborne-mediated microbial diseases such as i...,airbornemedi microbi diseas influenza tubercul...,welch buonanno grilj shuryak crickmor bigelow ...


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [144]:
PATH_QUERY_TRAIN_DATA = 'subtask_4b/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA = 'subtask_4b/subtask4b_query_tweets_dev.tsv'

In [145]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [146]:
df_query_train['normalized_tweet_text'] = df_query_train['tweet_text'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)


df_query_dev['normalized_tweet_text'] = df_query_dev['tweet_text'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)
df_query_train.head()


,post_id,tweet_text,cord_uid,normalized_tweet_text
0,16,covid recovery: this study from the usa reveal...,3qvh482o,covid recovery: this study from the usa reveal...
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"""among 139 clients exposed to two symptomatic ..."
2,73,I recall early on reading that researchers who...,sts48u9i,i recall early on reading that researchers wor...
3,93,You know you're credible when NIH website has ...,3sr2exq9,you know you're credible when national institu...
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,resistance to antifungal medications is a grow...


In [150]:
df_query_train['cleaned_tweet_text'] = df_query_train['tweet_text'].parallel_apply(preprocess_text)
df_query_train['cleaned_tweet_text'] = df_query_train['cleaned_tweet_text'].fillna('').astype(str)

df_query_dev['cleaned_tweet_text'] = df_query_dev['tweet_text'].parallel_apply(preprocess_text)
df_query_dev['cleaned_tweet_text'] = df_query_dev['cleaned_tweet_text'].fillna('').astype(str)


# 2)The following code runs a BM25 after preprcoessing



In [160]:
from rank_bm25 import BM25Plus

corpus = df_collection[['cleaned_title', 'cleaned_abstract', 'cleaned_authors']].parallel_apply(
    lambda x: f"{x['cleaned_title']} {x['cleaned_abstract']} {x['cleaned_authors']}",
    axis=1
).tolist()

cord_uids = df_collection[:]['cord_uid'].tolist()

tokenized_corpus = [doc.split(' ') for doc in corpus]

bm25 = BM25Plus(tokenized_corpus)

The relevant documents are retrived using the BM25 model.

In [161]:
text2bm25top = {}
def get_top_cord_uids(query):
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')


      #based on the tokenized query we call the BM25 to compute the relevance score for each document relevant to the query
      doc_scores = bm25.get_scores(tokenized_query)
      # it sort the doc scores in descending order
      indices = np.argsort(-doc_scores)[:5]
      #This line uses the indices from the previous step to retrieve the actual document IDs
      bm25_topk = [cord_uids[x] for x in indices]


      text2bm25top[query] = bm25_topk
      return bm25_topk

In [162]:
df_query_train['final_query'] = df_query_train['cleaned_tweet_text']
df_query_dev['final_query'] = df_query_dev['cleaned_tweet_text']

df_query_train.head()

,post_id,tweet_text,cord_uid,normalized_tweet_text,cleaned_tweet_text,final_query
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,oral care in rehabilitation medicine: oral vul...,oral care rehabilit medicin oral vulner oral m...,oral care rehabilit medicin oral vulner oral m...
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,this study isn't receiving sufficient attentio...,studi isnt receiv suffici attent reveal blackl...,studi isnt receiv suffici attent reveal blackl...
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"thanks, xi jinping. a reminder that this study...",thank xi jinp remind studi conclud nonpharmace...,thank xi jinp remind studi conclud nonpharmace...
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,taiwan - a population of 23 million has had ju...,taiwan popul 23 million 600 case 7 death wides...,taiwan popul 23 million 600 case 7 death wides...
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,obtaining a diagnosis of autism in lower incom...,obtain diagnosi autism lower incom countri tak...,obtain diagnosi autism lower incom countri tak...


In [163]:
df_query_train['bm25_topk'] = df_query_train['final_query'].parallel_apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['final_query'].parallel_apply(lambda x: get_top_cord_uids(x))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [164]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [165]:
# Evaluate retrieved candidates using MRR@k
#results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')

# Printed MRR@k results
print(f"Results on the train set: {dict((k, float(v)) for k, v in results_train.items())}")
print(f"Results on the dev set: {dict((k, float(v)) for k, v in results_dev.items())}")


Results on the train set: {1: 0.5644596592235276, 5: 0.6171646048912056, 10: 0.6171646048912056}
Results on the dev set: {1: 0.5657142857142857, 5: 0.616095238095238, 10: 0.616095238095238}


In [166]:
def results_to_markdown_table(train_results, dev_results):
    # Header
    table = "| Set   | Top-K | Score     |\n"
    table += "|--------|--------|------------|\n"

    # Train rows
    for k, score in train_results.items():
        table += f"| Train | {k}     | {score:.4f} |\n"

    # Dev rows
    for k, score in dev_results.items():
        table += f"| Dev   | {k}     | {score:.4f} |\n"

    return table

print(results_to_markdown_table(results_train, results_dev))

| Set   | Top-K | Score     |
|--------|--------|------------|
| Train | 1     | 0.5645 |
| Train | 5     | 0.6172 |
| Train | 10     | 0.6172 |
| Dev   | 1     | 0.5657 |
| Dev   | 5     | 0.6161 |
| Dev   | 10     | 0.6161 |



| Set   | Top-K | Score     |
|--------|--------|-----------|
| Train | 1     | 0.5221 |
| Train | 5     | 0.5747 |
| Train | 10    | 0.5747 |
| Dev   | 1     | 0.5186 |
| Dev   | 5     | 0.5740 |
| Dev   | 10    | 0.5740 |


# 4) Evaluating different spaCy models

In [ ]:
!pip install -U scispacy

Python(96810) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 84.3 kB/s eta 0:00:00 0:00:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 76.7 kB/s eta 0:00:0000:0100:04

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
def compare_spacy_models(candidate_models):
    results = {}

    try:
        for model_name in candidate_models:
            print(f"Testing model: {model_name}")

            # Install only if needed
            try:
                if not spacy.util.is_package(model_name):
                    print(f"Installing {model_name}...")
                    spacy.cli.download(model_name)
                terms_extractor = spacy.load(model_name)
            except Exception as e:
                print(f"❌ {model_name} failed: {e}")
                continue

            # Process a sample with the new model
            df_query_sample = df_query_train.head(100).copy()
            df_query_sample['cleaned_tweet_text'], df_query_sample['entities_in_tweet_text'] = zip(
                *df_query_sample['tweet_text'].parallel_apply(
                    lambda x: preprocess_text(x, terms_extractor)
                )
            )

            # Update get_top_cord_uids for this test
            df_query_sample['final_query'] = df_query_sample['cleaned_tweet_text'] + ' ' + df_query_sample['entities_in_tweet_text']
            df_query_sample['bm25_topk'] = df_query_sample['final_query'].parallel_apply(lambda x: get_top_cord_uids(x))

            # Evaluate
            mrr_result = get_performance_mrr(df_query_sample, 'cord_uid', 'bm25_topk')
            results[model_name] = mrr_result
    except Exception as e:
        print(f"Failed: {e}")

    # Display results as a table
    df_results = pd.DataFrame(results).T
    return df_results

In [ ]:
models = [
    "en_core_web_sm",
    "en_core_web_md",
    "en_core_web_lg",
    # "en_core_web_trf",  # problems with some dependencies
    "en_core_sci_md",
    # "en_core_sci_lg"  # the process is getting killed
]
compare_spacy_models(models)

Testing model: en_core_web_sm


Python(96945) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(96946) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96947) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96948) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96949) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96950) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96951) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96952) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96953) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96954) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(96955) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96956) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96957) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96958) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96959) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96960) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96961) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(96962) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Testing model: en_core_web_md


...

Testing model: en_core_web_trf
❌ en_core_web_trf failed: [E002] Can't find factory for 'curated_transformer' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, en.lemmatizer
Testing model: en_core_sci_md


/Users/daniel.levin/Library/Mobile Documents/com~apple~CloudDocs/MA/Semester_2/Advanced Information Retrieval/AIR_SS25/venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_md' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Python(97037) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(97038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97040) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97041) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97042) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97043) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97045) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97474) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(97475) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97476) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97477) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97478) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97479) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97480) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97482) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(97483) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


,1,5,10
en_core_web_sm,0.57,0.621500,0.621500
en_core_web_md,0.58,0.621667,0.621667
en_core_web_lg,0.58,0.611500,0.611500
en_core_sci_md,0.55,0.616167,0.616167


# 5) Exporting results to prepare the submission on Codalab

In [ ]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [ ]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')